# Demo Notebook

<a target="_blank" href="https://colab.research.google.com/github/s2t2/gspread-models-py/blob/main/docs/notebooks/demo_v1_0_6.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

This is a Google Colab notebook that showcases various features of the [`gspread-models` package](https://github.com/s2t2/gspread-models-py).

In this demo, we take advantage of the simplified Google authentication process within Colab to obtain credentials for the logged in user (instead of using a service account credentials JSON file).

## Package Installation

Package installation:

In [1]:
%%capture
#!pip uninstall gspread_models
!pip install git+https://github.com/s2t2/gspread-models-py.git@binding
#!pip install gspread_models==1.0.6

In [5]:
#!pip list | grep gspread_models
!pip list | grep gspread-models

gspread-models                   1.0.6


In [7]:
from gspread_models import VERSION
print(VERSION)

1.0.6


## Google Credentials

In [8]:
from google.colab import auth

auth.authenticate_user()

In [9]:
from google.auth import default

creds, _ = default()
type(creds)

google.auth.compute_engine.credentials.Credentials

## Spreadsheet Service

Create a Google Sheets document with "books" sheet, with columns: `id`, `title`, `author`, `year`, and `created_at`. Note the document identifier, and set it as the `GSPREAD_MODELS_DOCUMENT_ID` notebook secret.

In [10]:
from google.colab import userdata

GOOGLE_SHEETS_DOCUMENT_ID = userdata.get("GSPREAD_MODELS_DOCUMENT_ID")

Build a new service instance:

In [11]:
from gspread_models.service import SpreadsheetService

service = SpreadsheetService(creds=creds, document_id=GOOGLE_SHEETS_DOCUMENT_ID)

SPREADSHEET SERVICE...
DOCUMENT ID: 1Nk-UBd-3TyWZqbWSvKIIF-S_KKdrkVFRVeNrFy_F1gs


In [12]:
service.doc

<Spreadsheet 'GSpread Models (Test Database)' id:1Nk-UBd-3TyWZqbWSvKIIF-S_KKdrkVFRVeNrFy_F1gs>

In [13]:
for sheet in service.sheets:
    print(sheet.title)

Info
products
orders
books


## Binding the Base Model

Bind the base model with access to your account credentials and your document:

In [14]:
from gspread_models.base import BaseModel

# if you already have the service handy, you can set it:
#BaseModel.service = service

# otherwise you can bypass the service altogether:
BaseModel.bind(credentials=creds, document_id=GOOGLE_SHEETS_DOCUMENT_ID)

SPREADSHEET SERVICE...
DOCUMENT ID: 1Nk-UBd-3TyWZqbWSvKIIF-S_KKdrkVFRVeNrFy_F1gs


## Defining Child Model Classes

Now, defining custom classes that inherit from the base model:

In [15]:
class Book(BaseModel):

    SHEET_NAME = "books"

    COLUMNS = ["title", "author", "year"]

## Queries

Destroy all records:

In [16]:
Book.destroy_all()

GET SHEET ('books')...


{'spreadsheetId': '1Nk-UBd-3TyWZqbWSvKIIF-S_KKdrkVFRVeNrFy_F1gs',
 'replies': [{}]}

Populate sheet with default records:

In [17]:
Book.create_all([
    {"title": "To Kill a Mockingbird", "author": "Harper Lee", "year": 1960},
    {"title": "1984", "author": "George Orwell", "year": 1949},
    {"title": "The Great Gatsby", "author": "F. Scott Fitzgerald", "year": 1925},
    {"title": "The Catcher in the Rye", "author": "J.D. Salinger", "year": 1951},
    {"title": "Pride and Prejudice", "author": "Jane Austen", "year": 1813},
    {"title": "To the Lighthouse", "author": "Virginia Woolf", "year": 1927},
    {"title": "The Hobbit", "author": "J.R.R. Tolkien", "year": 1937},
    {"title": "Moby-Dick", "author": "Herman Melville", "year": 1851},
    {"title": "Brave New World", "author": "Aldous Huxley", "year": 1932},
    {"title": "Alice's Adventures in Wonderland", "author": "Lewis Carroll", "year": 1865},
    {"title": "Harry Potter and the Philosopher's Stone", "author": "J.K. Rowling", "year": 1997},
    {"title": "Harry Potter and the Chamber of Secrets", "author": "J.K. Rowling", "year": 1998},
])

{'spreadsheetId': '1Nk-UBd-3TyWZqbWSvKIIF-S_KKdrkVFRVeNrFy_F1gs',
 'tableRange': 'books!A1:F1',
 'updates': {'spreadsheetId': '1Nk-UBd-3TyWZqbWSvKIIF-S_KKdrkVFRVeNrFy_F1gs',
  'updatedRange': 'books!A2:E13',
  'updatedRows': 12,
  'updatedColumns': 5,
  'updatedCells': 60}}

Fetch all records from the sheet:

In [18]:
books = Book.all()
len(books)

12

In [19]:
for book in books:
    print(book.id, book.title, book.author, sep=" | ")
    # book.created_at.strftime("%Y-%m-%d")

1 | To Kill a Mockingbird | Harper Lee
2 | 1984 | George Orwell
3 | The Great Gatsby | F. Scott Fitzgerald
4 | The Catcher in the Rye | J.D. Salinger
5 | Pride and Prejudice | Jane Austen
6 | To the Lighthouse | Virginia Woolf
7 | The Hobbit | J.R.R. Tolkien
8 | Moby-Dick | Herman Melville
9 | Brave New World | Aldous Huxley
10 | Alice's Adventures in Wonderland | Lewis Carroll
11 | Harry Potter and the Philosopher's Stone | J.K. Rowling
12 | Harry Potter and the Chamber of Secrets | J.K. Rowling


Find record by id:

In [20]:
book = Book.find(3)
dict(book)

{'id': 3,
 'title': 'The Great Gatsby',
 'author': 'F. Scott Fitzgerald',
 'year': 1925,
 'created_at': datetime.datetime(2024, 5, 23, 0, 31, 9, 353691, tzinfo=datetime.timezone.utc)}

Filter records on matching conditions:


In [21]:
book = Book.where(author="F. Scott Fitzgerald")[0]
dict(book)

{'id': 3,
 'title': 'The Great Gatsby',
 'author': 'F. Scott Fitzgerald',
 'year': 1925,
 'created_at': datetime.datetime(2024, 5, 23, 0, 31, 9, 353691, tzinfo=datetime.timezone.utc)}

In [22]:
hp_books = Book.where(author="J.K. Rowling")
for book in hp_books:
    print(book.id, book.title, book.author, sep=" | ")

11 | Harry Potter and the Philosopher's Stone | J.K. Rowling
12 | Harry Potter and the Chamber of Secrets | J.K. Rowling


## Pandas Support

### Formatting Records as Pandas DataFrame

FYI: It is easy to construct a pandas DataFrame containing the information fetched from the sheet, by converting the records to a list of dictionaries:

In [23]:
from pandas import DataFrame

books_df = DataFrame([dict(book) for book in books])
books_df.head()

,id,title,author,year,created_at
0,1,To Kill a Mockingbird,Harper Lee,1960,2024-05-23 00:31:09.352598+00:00
1,2,1984,George Orwell,1949,2024-05-23 00:31:09.353662+00:00
2,3,The Great Gatsby,F. Scott Fitzgerald,1925,2024-05-23 00:31:09.353691+00:00
3,4,The Catcher in the Rye,J.D. Salinger,1951,2024-05-23 00:31:09.353712+00:00
4,5,Pride and Prejudice,Jane Austen,1813,2024-05-23 00:31:09.353748+00:00


### Customizing Base Model Behavior

If you would like to build additional pandas functionality into your child classes, you can leverage inheritence to overwrite methods of the base model:

In [24]:

from pandas import DataFrame

class MyBaseModel(BaseModel):

    @classmethod
    def all(cls, as_df=False):
        """Fetches all records from a given sheet.

            Overwrites parent method to optionally use a dataframe approach.

            Params : as_df (bool) : will return data as a DataFrame
        """
        records = cls.sheet.get_all_records()
        if as_df:
            # use model conversion to take advantage of datetime-conversion, etc:
            df = DataFrame([dict(cls(record)) for record in records])
            df.index = df["id"]
            return df
        else:
            return [cls(record) for record in records]


MyBaseModel.service = service

Now all child models will have this capability:

In [25]:
class MyBook(MyBaseModel):

    SHEET_NAME = "books"

    COLUMNS = ["title", "author", "year"]


In [26]:
books_df = MyBook.all(as_df=True)
books_df.head()

GET SHEET ('books')...


,id,title,author,year,created_at
id,,,,,
1,1,To Kill a Mockingbird,Harper Lee,1960,2024-05-23 00:31:09.352598+00:00
2,2,1984,George Orwell,1949,2024-05-23 00:31:09.353662+00:00
3,3,The Great Gatsby,F. Scott Fitzgerald,1925,2024-05-23 00:31:09.353691+00:00
4,4,The Catcher in the Rye,J.D. Salinger,1951,2024-05-23 00:31:09.353712+00:00
5,5,Pride and Prejudice,Jane Austen,1813,2024-05-23 00:31:09.353748+00:00
